In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
assert tf.__version__ >= "2.7.0"

In [ ]:
tf.config.run_functions_eagerly(True)
tf.executing_eagerly()
tf.data.experimental.enable_debug_mode()

In [ ]:
# Model / data parameters
num_classes = 20
input_shape = (32, 32, 3)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='coarse')

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to one-hot representations
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

169017344/169001437 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
@tf.custom_gradient
def generate_B(W,u): # generate B given weight W and a specified u
  def grad(upstream): # define gradient by ourselves (dc/dW, dc/du)
    dB_dW = 1
    return upstream * dB_dW, 1
  std_W = tf.math.reduce_std(W) # calculate std(W)
  mean_W =  tf.math.reduce_mean(W) # calculate mean(W)
  Bi = tf.math.sign(W - mean_W + u * mean_W) #calculate Bi
  return Bi, grad

In [ ]:
from sklearn.linear_model import LinearRegression
def generate_alpha(W,list_of_B): # generate alpha given weight W and list of B containing all Bi
  W = tf.reshape(W, [-1]).numpy() #vectorize W
  list_of_reshape_B = []
  for i in list_of_B: # vectorize Bi
    list_of_reshape_B.append(tf.reshape(i, [-1]))
  B = tf.stack(list_of_reshape_B, axis=1).numpy() # put Bi together into a large matrix
  reg = LinearRegression(fit_intercept=False).fit(B, W) # linear regression
  alpha = reg.coef_ # coefficient of linear regression, which is alpha
  return alpha

In [ ]:
@tf.custom_gradient
def H_v(R, v): # calculate H_v given input R and shift parameter v
  def grad(upstream):  # define gradient by ourselves (dc/dR, dc/dv)
    return upstream, tf.math.reduce_sum(upstream)
  return 2 * tf.cast(tf.math.greater(tf.math.add(R, v), tf.constant(0.5)), tf.float32) - 1, grad # calculate H_v

Initialization of ABCConv class with M=3,N=1

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers
import numpy as np

# number of binary activations
N = 1
class ABCConv(layers.Conv2D): # subclass of layers.Conv2D
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.betas = self.add_weight(
            shape=(N,), initializer="random_normal", trainable=True
        )
        self.vs = self.add_weight(shape=(N,), initializer="random_normal", trainable=True)

    def convolution_op(self, inputs, kernel): # override convolution_op
        ## Multiple binary activations and bitwise convolution
        
        # given beta's and v's, get all binary activations
        binary_activations = []
        list_of_A = []
        for i in range(0,N):
          list_of_A.append(H_v(inputs,self.vs[i]))
        

        # get all B
        M = 3
        list_of_u = []
        for i in range(1,M+1): # all u
          list_of_u.append(-1 + (i -1.0) * 2 / (M-1))
        list_of_B = []
        for i in range(1,M+1): # all B
          list_of_B.append(generate_B(kernel, list_of_u[i-1]))
       
        
        
        # get alphas 
        alphas = generate_alpha(kernel,list_of_B)
        # call individual convs
        convs = []
        for j in range(N):
          for i in range(M):
            convs.append(alphas[i]* self.betas[j] * tf.nn.conv2d(
                list_of_A[j],
                list_of_B[i],
                padding="VALID",
                strides=list(self.strides),
                name=self.__class__.__name__ + "conv" + str(i) + str(j),
              ))
        return tf.add_n(convs) # linear combination of all convolution results
        

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

model_3_1_VGG = keras.Sequential(
    [
        keras.Input(shape=input_shape),


        ABCConv(64, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        ABCConv(64, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        
        ABCConv(128, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        ABCConv(128, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        ABCConv(256, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        ABCConv(256, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        ABCConv(256, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),

        ABCConv(512, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        ABCConv(512, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        ABCConv(512, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        ABCConv(512, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        ABCConv(512, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        ABCConv(512, kernel_size=(3, 3), activation=None,  padding = "same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
             
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ]
)

model_3_1_VGG.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 abc_conv_52 (ABCConv)       (None, 32, 32, 64)        1794      
                                                                 
 batch_normalization_52 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 abc_conv_53 (ABCConv)       (None, 32, 32, 64)        36930     
                                                                 
 batch_normalization_53 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                      

In [ ]:
# M=3, N=1
batch_size = 64
epochs = 10

model_3_1_VGG.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.TopKCategoricalAccuracy(k=3)],run_eagerly=True)

model_3_1_VGG.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model_3_1_VGG.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Epoch 1/10
704/704 [==============================] - 922s 1s/step - loss: 3.0106 - categorical_accuracy: 0.0492 - top_k_categorical_accuracy: 0.1487 - val_loss: 2.9963 - val_categorical_accuracy: 0.0476 - val_top_k_categorical_accuracy: 0.1406
Epoch 2/10
704/704 [==============================] - 918s 1s/step - loss: 2.9959 - categorical_accuracy: 0.0499 - top_k_categorical_accuracy: 0.1483 - val_loss: 2.9964 - val_categorical_accuracy: 0.0422 - val_top_k_categorical_accuracy: 0.1352
Epoch 3/10
704/704 [==============================] - 912s 1s/step - loss: 2.9959 - categorical_accuracy: 0.0492 - top_k_categorical_accuracy: 0.1479 - val_loss: 2.9964 - val_categorical_accuracy: 0.0422 - val_top_k_categorical_accuracy: 0.1342
Epoch 4/10
704/704 [==============================] - 911s 1s/step - loss: 2.9959 - categorical_accuracy: 0.0499 - top_k_categorical_accuracy: 0.1492 - val_loss: 2.9967 - val_categorical_accuracy: 0.0422 - val_top_k_categorical_accuracy: 0.1336
Epoch 5/10
704/704 [